In [1]:
# Distruct 3: 1.275896	103.812776
# District 7: 1.295695	103.856892
# District 20: 1.366447	103.854150

In [2]:
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv 

In [3]:
token = getpass()

In [3]:
#Schools 1000m
url = "https://api.foursquare.com/v3/places/search?query=school&ll=1.366447%2C103.854150&radius=1000&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token) 
}

response = requests.get(url, headers=headers)

print(response.text)

#starbucks 750m

url = "https://api.foursquare.com/v3/places/search?ll=1.366447%2C103.854150&radius=750&chains=ab4c54c0-d68a-012e-5619-003048cad9da&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token)
}

response = requests.get(url, headers=headers)

# Night clubs within 1km
url = "https://api.foursquare.com/v3/places/search?ll=1.366447%2C103.854150&radius=1000&categories=10032&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token)
}

response = requests.get(url, headers=headers)

print(response.text)

#Vegan restaurant within 500m

url = "https://api.foursquare.com/v3/places/search?ll=1.366447%2C103.854150&radius=500&categories=13377&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token)
}

response = requests.get(url, headers=headers)

# Basketball court within 1km

url = "https://api.foursquare.com/v3/places/search?ll=1.366447%2C103.854150&radius=1000&categories=18008&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token)
}

response = requests.get(url, headers=headers)

print(response.text)

# Pet grooming services witin 1km

url = "https://api.foursquare.com/v3/places/search?ll=1.366447%2C103.854150&radius=1000&categories=11134&limit=10"

headers = {
    "accept": "application/json",
    "Authorization": (token)
}

response = requests.get(url, headers=headers)

print(response.text)



{"results":[{"fsq_id":"4cb588f17abcbfb737e7a51f","categories":[{"id":12059,"name":"High School","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/building/school_","suffix":".png"}}],"chains":[],"distance":188,"geocodes":{"main":{"latitude":1.365972,"longitude":103.852415},"roof":{"latitude":1.365972,"longitude":103.852415}},"link":"/v3/places/4cb588f17abcbfb737e7a51f","location":{"address":"1 Ang Mo Kio St 42","country":"SG","formatted_address":"1 Ang Mo Kio St 42, 569277","locality":"Singapore","post_town":"Ang Mo Kio","postcode":"569277"},"name":"Deyi Secondary School","related_places":{},"timezone":"Asia/Singapore"},{"fsq_id":"5420cbfd498e3d0290428fce","categories":[{"id":12053,"name":"Language School","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/education/default_","suffix":".png"}}],"chains":[],"distance":344,"geocodes":{"main":{"latitude":1.368998,"longitude":103.856131},"roof":{"latitude":1.368998,"longitude":103.856131}},"link":"/v3/places/5420cbfd498e3d0290

In [8]:
def get_foursquare_data(lat, long):
    base_url = "https://api.foursquare.com/v3/places/search"
    links = [
        f"query=school&ll=1.366447%2C103.854150&radius=1000&limit=10",
        f"ll=1.366447%2C103.854150&radius=750&chains=ab4c54c0-d68a-012e-5619-003048cad9da&limit=10",
        f"ll=1.366447%2C103.854150&radius=1000&categories=10032&limit=10",
        f"ll=1.366447%2C103.854150&radius=500&categories=13377&limit=10",
        f"ll=1.366447%2C103.854150&radius=1000&categories=18008&limit=10",
        f"ll=1.366447%2C103.854150&radius=1000&categories=11134&limit=10"

    ]
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    counts = []
    for link in links:
        url = f"{base_url}?{link}"
        response = requests.get(url, headers=headers)
        count = len(response.json().get("results", []))
        counts.append(count)
    return pd.DataFrame({"Category": ["Schools", "Starbucks", "Night Clubs", "Vegan Restaurants", "Basketball Courts", "Pet Grooming Services"], "Count": counts})

In [9]:
get_foursquare_data(1.275896,103.812776)

# Distruct 3: 1.275896,103.812776
# District 7: 1.295695	103.856892
# District 20: 1.366447	103.854150

,Category,Count
0,Schools,10
1,Starbucks,0
2,Night Clubs,2
3,Vegan Restaurants,6
4,Basketball Courts,3
5,Pet Grooming Services,3
